In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

from datetime import datetime
from datetime import date

In [3]:
df_model = pd.read_csv("../inter_data/preprocessed.csv")

df_model = df_model[[col for col in df_model.columns if "Unnamed" not in col]]

df_model["target"] = df_model["num_referees_trading"]/df_model["refer_count"]







In [4]:
def feature_engineering(df):
   
    # handing null values(continuos values)
    del df["cm_cd"]
    del df["cm_mobile"]
    del df["activation_count"]
    del df["num_referees_trading"]
    del df["refer_count"]
    del df["num_referees_logging_in"]
    df["months_since_last_login"] = df["months_since_last_login"].fillna(df["months_since_last_login"].mean())
    df["months_since_last_trade"] = df["months_since_last_trade"].fillna(df["months_since_last_trade"].mean())

     
    # categorical data
    df["KRA YN"] = df["KRA YN"].replace({"Y": 1, 
                                        "N": 0})
    df["KRA YN"] = df["KRA YN"].fillna(0)
    
    
    e2channels = list(df["E2Channel"].value_counts().index[:5])
    df['E2Channel'].loc[~df['E2Channel'].isin(e2channels)] = 'others'
    
    income_slabs = list(df["IncomePA"].value_counts().index[:5])
    df["IncomePA"] = df["IncomePA"].replace({"5_10L ??": "5_10L", 
                                        "<=1??": "<=1L", 
                                        "1_5??": "1_5L", "<= 1L": "<=1L"})
    df['IncomePA'].loc[~df['IncomePA'].isin(income_slabs)] = 'others'

    df["IncomePA"] = df["IncomePA"].map({"others": 0, "<=1L": 1, "1lakh": 2, "1_2L": 3, "1_5L": 3,"2_5L": 3, ">5L": 4, "5_10L": 4, "10_25L": 5, ">25:": 6})
    
    df['IsKarvy'] = df['IsKarvy'].fillna(0)
    
    
    df["IncomePA"] = df["IncomePA"].fillna(0)
    # creating dummy for categorical data
    dummy1 = pd.get_dummies(df[["E2Channel", "cm_occup"]], drop_first=True)
    df = pd.concat([df, dummy1], axis=1)
    df = df.drop(["E2Channel", "cm_occup"], axis = 1)
   
   
    # scaling numerical columns
    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    numerical_cols = ["age","months_since_last_login","months_since_last_trade", "total_logins_one_year", 
                      "total_trades_one_year", "referees_avg_logins_in_last_year", "referees_avg_trades_in_last_year"]
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

    df[numerical_cols] = df[numerical_cols].fillna(0)

    return df


In [5]:
def feature_engineering_catboost(df):
   
    # handing null values(continuos values)
    del df["cm_cd"]
    del df["cm_mobile"]
    del df["activation_count"]
    del df["num_referees_trading"]
    del df["refer_count"]
    del df["num_referees_logging_in"]
    df["months_since_last_login"] = df["months_since_last_login"].fillna(df["months_since_last_login"].mean())
    df["months_since_last_trade"] = df["months_since_last_trade"].fillna(df["months_since_last_trade"].mean())

     
    
    
    
    
   
    # scaling numerical columns
    from sklearn.preprocessing import StandardScaler

    numerical_cols = ["age","months_since_last_login","months_since_last_trade", "total_logins_one_year", 
                      "total_trades_one_year", "referees_avg_logins_in_last_year", "referees_avg_trades_in_last_year"]

    df[numerical_cols] = df[numerical_cols].fillna(0)

    return df


In [159]:
df_model2 = feature_engineering_catboost(df_model.copy())
df_model3 = feature_engineering(df_model.copy()) 


In [160]:
df_model2.to_csv("../inter_data/processed_catboost.csv")
df_model3.to_csv("../inter_data/processed_encoded.csv")